# Creating a custom Index in Pandas

The pandas.Datetimeindex is the jam because of the extra properties that access portions of the time.  I'd really like a `pathlib` version of this, `...` and a `luigi` target version.

In [1]:
from pathlib import Path, PurePath, os
from pandas import Index, Series, DataFrame; import pandas as pd
from toolz.curried.operator import *; from toolz.curried import *

## Using typing information as data

`pathlib` has an annotated type file in `typeshed`.  The script below makes the annotations importable.

In [2]:
def get_types(file):
    """Search for typeshed files in the sys path; requires mypy."""
    assert str(file).endswith('.pyi')
    locals().update(globals()) or exec(next(Path(__import__('sys').prefix).rglob(file)).read_text(), locals())
    return __import__('argparse').Namespace(**locals())

types = get_types('pathlib.pyi')
target = types.Path

The  __annotations__ for `Path`'s attributes are held in `target`; _this is a function parameter later_.

In [3]:
def method(x, target=target):
    """Get annotations from bound methods and properties.s"""
    if isinstance(x, property):
        return getattr(target, x.fget.__name__).fget.__annotations__['return']
    attr = getattr(x, '__name__', None)
    if attr in dir(target):
        return getattr(getattr(target, attr), '__annotations__', {}).get('return', None)

`returns` is an object of the attributes and their return `type`.

In [4]:
returns = pipe(
    target, dir, filter(compose(str.isalpha, first)), 
    map(juxt(identity, curry(getattr)(target))),
    dict, valmap(method))

Series(returns).astype(str).to_frame().reset_index().set_index(0).sort_index().sample(10).T

,<class 'os.stat_result'>,None,None,<class 'str'>,"typing.Generator[pathlib.Path, NoneType, NoneType]",<class 'bool'>,None,None,<class 'bool'>,None
index,lstat,rename,rmdir,read_text,glob,is_symlink,unlink,root,samefile,chmod


> A small sample of the type to attribute mapping.

In [5]:
from functools import wraps

class PathIndex(Index):
    def __new__(cls, data=None, dtype=None, copy=False, name=None, fastpath=False, tupleize_cols=True, **kwargs):
        if isiterable(data) and not isinstance(data, Index):
            data = cls._coerce_to_ndarray(map(Path, data))
        result = cls._simple_new(data, name, dtype, copy=copy, fastpath=fastpath, tupleize_cols=tupleize_cols, **kwargs)
        return result
    
    def __getattribute__(self, name):
        try:
            return super().__getattribute__(name)
        except AttributeError as e:
            attr = getattr(Path, name, None)
            
            # The name is wrong!
            if not attr: raise e
            
            cls = type_mapper[method(attr)]
            
            # There is no way to interpret the attribute
            if cls is False: raise NotImplemented
            
            
            if isinstance(attr, property): #  return computed values, not a callable.
                return (self if cls is None else cls(self.map(attrgetter(name))))

            # wrap a callable for later.
            @wraps(attr)
            def mapper(*args, **kwargs):                        
                if cls is None: return self        
                result = cls(self.map(partial(lambda v: attr(v, *args, **kwargs))))
                if isinstance(result, (Series, DataFrame)):
                    result.index = self
                return result
            return mapper
            
    def __truediv__(self, value):
        return type(self)(self.map(flip(Path.__truediv__)(value)))
    
    def __rtruediv__(self, value):
        return type(self)(self.map(flip(Path.__rtruediv__)(value)))

    def __dir__(self):
        """Update the available attribute."""
        return list(super().__dir__()) + dir(Path)

In [6]:
type_mapper = {
    bool: Index,
    types._P: PathIndex,
    None: None, 
    types.IO[types.Any]: False,
    types.Generator[Path, None, None]: compose(PathIndex, list, concat),
    types.Sequence[types._P]: compose(PathIndex, list, concat),
    int: Index, os.stat_result: Index,
    str: Series, bytes: Series,
}

## A test drive of `PathIndex`


In [7]:
assert all(map(contains(type_mapper.keys()), returns.values()))

if __name__ == '__main__':
    i = PathIndex(['nbd', 'fidget', '.'])
    print(i.exists(), ('../..'/i).exists())

Index([False, False, True], dtype='object') Index([True, True, True], dtype='object')


In [8]:
if __name__ == '__main__':
    df = ('../..'/i)[:2].glob('*.ipynb').read_text().apply(
        __import__('nbformat').reads, args=[4]
    ).apply(
        compose(DataFrame, get('cells'))
    ).pipe(compose(pd.concat, dict, Series.iteritems))
    print(i.exists(), ('../..'/i).exists())
__name__ == '__main__' and df.sample(5)

Index([False, False, True], dtype='object') Index([True, True, True], dtype='object')


,,cell_type,execution_count,metadata,outputs,source
../../nbd/readme.ipynb,9,markdown,NaN,{},NaN,## Developer\n\nConfigure the documentation fo...
../../fidget/testing.ipynb,6,code,7.0,{},"[{'output_type': 'execute_result', 'execution_...","x_()(10)\nx_()(10, 20)\nx_()(10, 20, 30)"
../../nbd/readme.ipynb,6,markdown,NaN,{},NaN,### 2. Execute `nbd`
../../fidget/readme.ipynb,2,markdown,NaN,{},NaN,`fidget` includes _most_ of the Python model a...
../../nbd/nbd.ipynb,2,code,2.0,{},[],from nbconvert.nbconvertapp import NbConvertAp...


### Notes

* Logic works.